In [ ]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit_algorithms.optimizers import COBYLA
import itertools
from collections import Counter

# 图着色问题参数
n = 4  # 顶点数
k = 3  # 颜色数
N = int(np.ceil(np.log2(k)))  # 每个顶点需要的比特数
P = 100  # 惩罚系数
num_qubits_binary = n * N  # 总量子比特数
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]  # 图的边


# 定义量子门
def append__z_term(qc, qubit, gamma):
    qc.rz(2 * gamma, qubit)


def append__zz_term(qc, qubit1, qubit2, gamma):
    qc.cx(qubit1, qubit2)
    qc.rz(2 * gamma, qubit2)
    qc.cx(qubit1, qubit2)


def append__mixer_term(qc, qubit, beta):
    qc.rx(-2 * beta, qubit)


# 成本哈密顿量 (Binary)
def get_cost_circuit_binary(gamma, qc):
    for i, j in edges:
        for k in range(N):
            coef = P / (2 ** N)  # 惩罚相邻顶点颜色相同
            append__zz_term(qc, i * N + k, j * N + k, gamma * coef)
    return qc


# 混合哈密顿量
def get_mixer_circuit(beta, qc):
    for i in range(qc.num_qubits):
        append__mixer_term(qc, i, beta)
    return qc


# 反转比特序
def invert_counts(s):
    return s[::-1]


# 成本函数 (Binary)
def cost_function_binary(params):
    beta, gamma = params[:len(params) // 2], params[len(params) // 2:]
    qc = QuantumCircuit(num_qubits_binary, num_qubits_binary)
    qc.h(range(num_qubits_binary))
    for i in range(len(beta)):
        qc = get_cost_circuit_binary(gamma[i], qc)
        qc = get_mixer_circuit(beta[i], qc)
    qc.measure(range(num_qubits_binary), range(num_qubits_binary))
    backend = Aer.get_backend('qasm_simulator')
    shots = 10000
    result = execute(qc, backend, seed_simulator=10, shots=shots).result()
    counts = result.get_counts(qc)

    expectation = 0
    H_opt = 0  # 最优解：无冲突
    threshold = P  # 允许 1 个冲突
    good_counts = 0

    for bitstring, count in counts.items():
        bitstring = invert_counts(bitstring)
        prob = count / shots
        x = np.array([int(b) for b in bitstring]).reshape(n, N)
        colors = [sum(2 ** k * x[i, k] for k in range(N)) for i in range(n)]
        conflicts = sum(1 for i, j in edges if colors[i] == colors[j])
        energy = P * conflicts
        expectation += energy * prob
        if energy <= threshold:
            good_counts += count

    opt_bitstring = min(counts.keys(), key=lambda b: cost_function_binary([params])[0])
    opt_prob = counts.get(opt_bitstring, 0) / shots
    r = expectation / (P * len(edges))  # 归一化
    good_prob = good_counts / shots

    return expectation, counts, opt_prob, r, good_prob, qc.depth()


# 参数优化
def optimize_qaoa(params, cost_func):
    optimizer = COBYLA(rhobeg=1.5, tol=1e-7)
    result = optimizer.minimize(fun=lambda p: cost_func(p)[0], x0=params)
    return result



# 主程序 (Binary)
import time

start_time = time.time()
min_energy_binary = float('inf')
final_result_binary = None
counts_binary = {}
metrics_binary = {}

for p1 in range(1, 4):
    for k in range(5):
        print(f"Binary - p={p1}, iter={k}")
        init_point = fourier_initial_params(p1, q=1, amplitude=0.5)
        result = optimize_qaoa(init_point, cost_function_binary)
        energy, counts_temp, opt_prob, r, good_prob, depth = cost_function_binary(result.x)
        print(f"Expectation: {energy}, P_opt: {opt_prob}, r: {r}, Good prob: {good_prob}, Depth: {depth}")
        if energy < min_energy_binary:
            min_energy_binary = energy
            final_result_binary = result
            counts_binary = counts_temp
            metrics_binary = {'energy': energy, 'opt_prob': opt_prob, 'r': r, 'good_prob': good_prob, 'depth': depth,
                              'qubits': num_qubits_binary}

end_time = time.time()
print(f"Binary - Total time: {end_time - start_time} s")
print(f"Binary - Final metrics: {metrics_binary}")

Binary - p=1, iter=0


C:\Users\李华宪\AppData\Local\Temp\ipykernel_56688\4169236379.py:56: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  result = execute(qc, backend, seed_simulator=10, shots=shots).result()


In [2]:
# import time
#
# # 运行优化
# startTime = time.time()
# ansatz_final = object()
# init_point = np.array([])
# counts = {}
# min_energy = 0
# final_result = object()
# for p1 in range(1, 4):
#     min_energy = 10000
#     for k in range(10):
#         print('第：', p1, '层', ' ', '第', k, '个参数')
#         init_point_temp = init_point
#         beta_p = np.random.uniform(0, np.pi)
#         gamma_p = np.random.uniform(0,  2 * np.pi)
#         if k >0:
#             init_point_temp = np.delete(init_point_temp, 2 * p1 - 1)
#             init_point_temp = np.delete(init_point_temp, p1 - 1)
#         init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
#         init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)
#         result = optimize_qaoa(init_point_temp)
#
#         # print(result.x)
#         optimal_params = result.x
#
#         # 计算最终的能量
#         energy, counts_temp = cost_function_onehot(result.x)
#         if min_energy > energy:
#             min_energy = energy
#             init_point = optimal_params
#             counts = counts_temp
#             final_result = result
# endTime = time.time()

第： 1 层   第 0 个参数


TypeError: optimize_qaoa() missing 1 required positional argument: 'cost_func'